In [1]:
import math
import heapq
import itertools
from copy import deepcopy
from threading import Lock
import pickle 
import numpy.random as random 
RUSH_HOUR_START = 6
RUSH_HOUR_END = 10 

In [67]:
class Graph:
    _graph = None
    _dist = None 
    _heuristic = None 
    _lock = None
    _lock_init = False


    @classmethod
    def dist(cls, N1,N2):
        x1, y1 = cls._graph[N1][0]
        x2, y2 = cls._graph[N2][0]
        return ((x1 - x2)**2 + (y1 - y2)**2)**(0.5)
    @classmethod 
    def mdist(cls, N1, N2): 
        x1, y1 = cls._graph[N1][0]
        x2, y2 = cls._graph[N2][0]
        return abs(x1-x2) + abs(y1-y2)

    heuristic = dist 
    
    @classmethod 
    def init(cls): 
        if not cls._lock_init: 
            cls._lock = Lock() 
            cls._lock_init = True 
    @classmethod
    def init_file(cls, file_string): 
        cls.init() 
        with open(file_string, 'rb') as f: 
            graph_dict, dist_dict = pickle.load(f) 
        cls._graph = graph_dict
        cls._dist = dist_dict
        # build total heuristic 
        cls._precompute_heuristic() 
   
    @classmethod 
    def _precompute_heuristic(cls): 
        cls._heuristic = {} 
        all_nodes = cls.get_nodes() 
        for i in range(len(all_nodes)): 
            for j in range(i, len(all_nodes)): 
                d = cls.heuristic(all_nodes[i], all_nodes[j])
                cls._heuristic[(all_nodes[i], all_nodes[j])] = d
                cls._heuristic[(all_nodes[j], all_nodes[i])] = d
                
    @classmethod
    def init_random(cls, N = 100, p = 0.1, max_weight = 100):
        '''
        Initialize a random graph with no planarity
        '''
        import random
        nodes = tuple(range(N))
        weights = list((1 + i for i in range(max_weight)))
        edges = []
        ones = [1] * N
        for i in range(N):
            ones[i] = 0
            new_edges = tuple(random.choices(nodes, weights=ones, k=int(N*p)))
            ones[i] = 1
            new_edge_weights = random.choices(weights, cum_weights= weights, k=int(N*p))
            new_dag_edges = tuple(zip(new_edges, new_edge_weights))
            edges.append(new_dag_edges)
        # generate random locations
        locations = [] 
        for i in range(N): 
            locations.append([random.random(), random.random()])

        # zip location, edges
        node_addr = list(zip(map(tuple, locations), edges))
        cls._graph = dict(zip(nodes, node_addr))

        # verify every node is reachable.
        checked_set = set()
        all_nodes = set(cls._graph.keys())
        for key, value in cls._graph.items():
            for edge in value[-1]:
                checked_set.add(edge[0]) # add all visited nodes to the checked_set
        difference_set = all_nodes.difference(checked_set)
        if len(difference_set) != 0:
            for node in difference_set:
                next_node = (node + 1) % N
                entry = cls._graph[next_node]
                edge_list = list(entry[-1])
                edge_list.append((node, 1))
                new_entry = [entry[0], edge_list]
                cls._graph[next_node] = new_entry
        cls._precompute_heuristic() 
    
    @classmethod
    def get_nodes(cls):
        return tuple(cls._graph.keys())

    @classmethod
    def get_neighbors(cls, node):
        return cls._graph[node][-1]

    @classmethod
    def compute_distance(cls, path):
        d = 0
        for i, node in enumerate(path):
            if i + 1 == len(path):
                break
            next_node = path[i+1]
            for e in cls.get_neighbors(node):
                if e[0] == next_node:
                    d += e[1]
                    break
        return d
    
    @classmethod
    def find_shortest_path(cls, start, end):
        with cls._lock: 
            # we're doing to do a* for now
            openset = []
            # save node, origin pairs
            path = dict()
            closedset = set()
            # openset contains f-score = weight + heuristic
            # closed set only contains node
            # we can just re-add to heap when node is encountered again
            # because min-heap, we will only infrequently encounter
            # the case when we reencounter a node
            # but we can check against closed set for that.
            push = heapq.heappush 
            pop = heapq.heappop 
            push(openset, (0, start, None))
            final_path = []
            found = False
            # while openset evaluates to T as long as openset is not empty
            while openset and not found:
                f, node, origin = pop(openset)

                path[node] = origin

                if node == end:
                    found = True
                    break
                
                if node in closedset:
                    continue # we already encountered it

                # generate successors
                edges = cls.get_neighbors(node)
                # add to open set
                for e in edges:
                    # don't go to nodes we hvae already visited
                    if e[0] in closedset:
                        continue
                    f_score = e[-1] + cls._heuristic[(e[0], end)]
                    push(openset, (f_score, e[0], node))
                closedset.add(node)
            c = end
            if found == False:
                return (-1, None)
            while c is not None:
                final_path.append(c)
                c = path[c]
            final_path = final_path[::-1]
            # compute distance
            total_distance = cls.compute_distance(final_path)
            return (total_distance, final_path)

    @classmethod
    def update_event(cls, requsted_time): 
        ''' not ready''' 
        local_copy = deepcopy(cls._graph) 
        interpolate_req = False 
        interp_weight_1 = 1 
        interp_weight_2 = 0
        start = 0 
        end = 0
        if requsted_time != int(requsted_time): 
            interpolate_req = True 
            start = math.floor(requsted_time)
            end = math.ceil(requsted_time)
            interp_weight_1 = requsted_time - start 
            interp_weight_2 = 1 - interp_weight_1
        for node in local_copy.keys(): 
            node_entry = local_copy[node] 
            edge_entry = node_entry[-1]
            new_edges = []
            edge_dist = None
            for neighbor, weight in edge_entry: 
                # grab distribution 
                if not interpolate_req: 
                    edge_dist = cls._dist[(node, neighbor)][requsted_time]
                else: 
                    edge_dist_1 = cls._dist[(node, neighbor)][start]
                    edge_dist_2 = cls._dist[(node, neighbor)][end] 
                    wa_mean = interp_weight_1 * edge_dist_1['mean'] + interp_weight_2 * edge_dist_2['mean']
                    wa_std  = math.sqrt((interp_weight_1 ** 2) * (edge_dist_1['stdev'] ** 2)  + (interp_weight_2**2)  * (edge_dist_2['stdev'] ** 2))
                    wa_gmean = interp_weight_1 * edge_dist_1['gmean'] + interp_weight_2 * edge_dist_2['gmean']
                    wa_gstdev = math.sqrt((interp_weight_1 ** 2) * (edge_dist_1['gstdev']**2) + (interp_weight_2 ** 2) * (edge_dist_2['gstdev'] ** 2))
                    edge_dist = {
                        "mean": wa_mean, 
                        "stdev": wa_std, 
                        "gmean": wa_gmean, 
                        "gstdev": wa_gstdev
                    }
                # draw from distribution 
                new_weight = random.normal(edge_dist['gmean'], edge_dist['gstdev']) 
                new_edges.append((neighbor, new_weight))
            local_copy[node] = (node_entry[0], tuple(new_edges))
        with cls._lock: 
            cls._graph = local_copy

In [68]:
g = Graph()
g.init_file("../pickles/graph.pypkle")


In [69]:
loc = g._graph
g.update_event(7.5)

In [70]:
g._dist[('8', '7')]

{6: {'mean': 244.5, 'stdev': 191.27, 'gmean': 187.39, 'gstdev': 2.1},
 10: {'mean': 307.17, 'stdev': 217.26, 'gmean': 242.37, 'gstdev': 2.06},
 8: {'mean': 395.61, 'stdev': 298.71, 'gmean': 295.99, 'gstdev': 2.25},
 9: {'mean': 354.55, 'stdev': 262.31, 'gmean': 274.08, 'gstdev': 2.14},
 7: {'mean': 318.21, 'stdev': 253.37, 'gmean': 235.36, 'gstdev': 2.26}}

In [71]:
g._graph['8']

((-77.01361182786752, 38.9008608020275),
 (('7', 268.1874581141062),
  ('223', 92.48240145757256),
  ('154', 168.7088204796864),
  ('186', 153.60322646699),
  ('45', 85.1274410105929)))

In [72]:
g._dist[('8', '7')]

{6: {'mean': 244.5, 'stdev': 191.27, 'gmean': 187.39, 'gstdev': 2.1},
 10: {'mean': 307.17, 'stdev': 217.26, 'gmean': 242.37, 'gstdev': 2.06},
 8: {'mean': 395.61, 'stdev': 298.71, 'gmean': 295.99, 'gstdev': 2.25},
 9: {'mean': 354.55, 'stdev': 262.31, 'gmean': 274.08, 'gstdev': 2.14},
 7: {'mean': 318.21, 'stdev': 253.37, 'gmean': 235.36, 'gstdev': 2.26}}